In [ ]:
#train_test_split is now in model_selection.
import os,cv2
import numpy as np 
import matplotlib.pyplot as plt

from sklearn.utils import shuffle 
from sklearn.model_selection import train_test_split 
    
from skimage import feature

from keras import backend as K 
K.set_image_dim_ordering('th') 
 
from keras.utils import np_utils 
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten 
from keras.layers.convolutional import Convolution2D, MaxPooling2D 
from keras.optimizers import SGD,RMSprop,Adam 
from sklearn.model_selection import StratifiedKFold 

In [ ]:
img_rows=300 
img_cols=300 
num_channel=1 
num_epoch=30 
 
# Define the number of classes 
num_classes = 8 

In [ ]:
Path = os.getcwd()
#num_classes = 8
dataset_path = "C:/Users/TA-D3TI-06 2018/Documents/Python Scripts/Dataset Ulos"
data = os.listdir(dataset_path)
img_data_list=[] 
kernel_sharpening = np.array([[-1,-1,-1], 
                              [-1, 9,-1],
                              [-1,-1,-1]])
for folder_dataset in data:
#     img_list = os.listdir(dataset_path+"/"+folder_dataset)
    img_list=os.listdir(dataset_path+"/"+ folder_dataset)  
    print ('Loaded the images of dataset-'+'{}\n'.format(folder_dataset))  
    for img in img_list:
        input_img=cv2.imread(dataset_path + "/"  + folder_dataset + "/" + img)
        input_img=cv2.cvtColor(input_img, cv2.COLOR_BGR2GRAY)
        input_img = cv2.GaussianBlur(input_img,(5,5),0)
        sharpened = cv2.filter2D(image, -1, kernel_sharpening)
        #Holisticly Edge Detection using Canny
        input_img=cv2.Canny(input_img, 10, 70)
        
        
        input_img_resize=cv2.resize(input_img,(img_rows,img_cols))   
        img_data_list.append(input_img_resize) 
 
img_data = np.array(img_data_list) 
img_data = img_data.astype('float32') 
img_data /= 255
print (img_data.shape) 

In [ ]:
if num_channel==1:  
    if K.image_dim_ordering()=='th':   
        img_data= np.expand_dims(img_data, axis=1)    
        print (img_data.shape)  
    else:   
        img_data= np.expand_dims(img_data, axis=4)    
        print (img_data.shape)    
else:  
    if K.image_dim_ordering()=='th':   
        img_data=np.rollaxis(img_data,3,1)   
        print (img_data.shape)    
        
# Assigning Labels 

num_of_samples = img_data.shape[0]
labels = np.ones((num_of_samples,),dtype='int64')

# =============================================================================
# names = ['Ulos Sibolang', 'Ulos Sadum', 'Ulos Ragi Hidup','Ulos Mangiring',
#'Ulos Harungguan', 'Ulos Ragi Hotang', 'Ulos Bintang Maratur', 'Ulos Sitolutuho']
# labels[0:632]=0
# labels[632:1440]=1
# labels[1440:2248]=2
# labels[2248:2912]=3
# labels[2912:3176]=4
# labels[3176:4272]=5
# labels[4272:4888]=6
# labels[4888:]=7
# # convert class labels to on-hot encoding
# =============================================================================

names = ['Ulos Bintang Maratur', 'Ulos Ragi Hidup', 'Ulos Sitolutuho','Ulos Sibolang', 
         'Ulos Ragi Hotang', 'Ulos Sadum', 'Ulos Harungguan', 'Ulos Mangiring']
labels[0:74]=0
labels[75:149]=1
labels[150:214]=2
labels[215:303]=3
labels[304:378]=4
labels[379:453]=5
labels[454:528]=6
labels[529:603]=7

Y = np_utils.to_categorical(labels, num_classes) 
#X = img_data
 
# #Shuffle the dataset 
x,y = shuffle(img_data,Y, random_state=2) 
seed = 7 
np.random.seed(seed) 
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed) 
 
#from sklearn.model_selection import train_test_split
#X_Train, X_Test, Y_Train, Y_Test = train_test_split(X, Y, train_size = 0.80, test_size = 0.20, random_state = 0)
#input_shape=img_data[0].shape 
#input_shape
#input_shape = input_shape + (1,)
#input_shape
#X_train = np.asarray( x[train_index]).reshape((len(X_train),360,380,1))
#X_Test = np.asarray(X_Test).reshape((len(X_Test),900,900,1))
#X_Train.shape

In [ ]:
input_shape = img_data[0].shape
counter = 0 

for train_index, test_index in kfold.split(x,y[:,0]):      
    print("TRAIN:", train_index, "\nTEST:", test_index)      
    X_train, X_test = x[train_index], x[test_index]      
    y_train, y_test = y[train_index], y[test_index]      
    counter+=1
    nilai_learning_rate=0.001
    opt=Adam(lr=nilai_learning_rate)
    np.savetxt('C:/Users/TA-D3TI-06 2018/Documents/Python Scripts/output VAR and Canny/'+str(counter)+'-train_index.csv', train_index, fmt='%d', delimiter=',')
    np.savetxt('C:/Users/TA-D3TI-06 2018/Documents/Python Scripts/output VAR and Canny/'+str(counter)+'-test_index.csv', test_index, fmt='%d', delimiter=',')
